In [1]:
import bioio

/home/marc/.miniconda/envs/bioio/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = bioio.load_biospec('biospec.yml')
dataset

<_ZipDataset element_spec={'outputs': {'SomeTask': {'total': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'control': TensorSpec(shape=(None,), dtype=tf.float32, name=None)}}}>

In [3]:
next(iter(dataset))

{'outputs': {'SomeTask': {'total': <tf.Tensor: shape=(201,), dtype=float32, numpy=
   array([  1.,   3.,   1.,   1.,   3.,   5.,   5.,   2.,  11.,   1.,   2.,
            6.,   5.,   2.,   2.,   0.,   2.,   2.,   0.,   0.,   0.,   0.,
            0.,   0.,   2.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,
           38., 119., 324., 103.,  32.,  29.,  18.,  44.,  15.,  23.,  37.,
            6.,   2.,   3.,   2.,   1.,   1.,   4.,   2.,   7.,   8.,   4.,
            1.,   3.,   2.,   4.,   1.,   0.,   0.,   6.,   0.,   1.,   1.,
     

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
tfds.dataset_builders.store_as_tfds_dataset(name='ENCODE_TIA1-HepG2', version='0.0.1', split_datasets = {'train': dataset}, features=features, data_dir='test_tfds')

In [ ]:
tfds.dataset_builders.TfDataBuilder(name='ENCODE_TIA1-HepG2', version='0.0.1', split_datasets = {'train': dataset}, data_dir='test_tfds')